In [ ]:
%matplotlib inline

In [ ]:
import functools
from typing import Tuple
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import networkx as nx
import numpy as np
import pandas as pd
from scipy import linalg
from scipy import spatial

# PHYS 395 - week 5

**Matt Wiens - #301294492**

This notebook will be organized similarly to the lab script, with major headings corresponding to the headings on the lab script.

*The TA's name (Ignacio) will be shortened to "IC" whenever used.*

## Setup

In [ ]:
# Set default plot size
plt.rcParams["figure.figsize"] = (12, 9)

In [ ]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999

# Vector/matrix operations

## Addition/subtraction

Note that the `+` operation when used on Python lists concatenates lists. However, for NumPy arrays, the `+` operation does elementwise addition.

In [ ]:
a_list = [1.1, -2.1, 0.0]
b_list = [2.0, 1.1, -0.5]

a = np.array(a_list)
b = np.array(b_list)

# + for lists
print("for lists: a + b = %s" % (a_list + b_list))

# + for NumPy arrays
print("for arrays: a + b = %s" % (a + b))

We can also do `-` for elementwise subtraction.

In [ ]:
print("a - b = %s" % (a - b))

## Multiplication by a scalar

Using NumPy arrays we can also do elementwise scalar multiplication. (Note that the `*` operator does not work between a float and a Python list.)

In [ ]:
v = np.array([1.0, -2.1, 3.0])

print(2.0 * v)

## Vector/matrix products

Using the `*` operator will perform elementwise multiplication (the $i$th element of the first array is multiplied by the $i$th element of the second array).

In [ ]:
print(a * b)

### Dot product

There are a few different ways of taking the dot product.

In [ ]:
# Using np.dot
print(np.dot(a, b))

# Using *
print(np.sum(a * b))

Using NumPy's `dot` function (or the `@` operator) allows us to calculate proper matrix products.

In [ ]:
sigma_x = np.array([[0, 1], [1, 0]])
sigma_z = np.array([[1, 0], [0, -1]])

Let's calculate $\sigma_x \sigma_z$.

In [ ]:
print(sigma_x @ sigma_z)

And $\sigma_z \sigma_x \sigma_z$.

In [ ]:
print(sigma_z @ sigma_x @ sigma_z)

### Vector norms

Calculating Euclidean norms is simple with NumPy. Here we'll calculate $| a |$.

In [ ]:
print(np.linalg.norm(a))

Or we determine the $|a|$ using the dot product.

In [ ]:
print(np.sqrt(np.dot(a, a)))

### Cross product

We can also easily compute cross products. Let's calculate $a \times b$.

In [ ]:
print(np.cross(a, b))

And let's verify this by evaluating the cross product ourselves:

\begin{align*}
    a \times b
        &= (a_y b_z - a_z b_y) \hat{i}
            + (a_z b_x - a_x b_z) \hat{j}
            + (a_x b_y - a_y b_x) \hat{z}
            \\
        &= ((-2.1) (-0.5) - (0) (1.1)) \hat{i}
            + ((0) (1.1) - (1.1) (-0.5)) \hat{j}
            + ((1.1) (1.1) - (-2.1) (2.0)) \hat{z}
            \\
        &= 1.05 \hat{i}
            + 0.55 \hat{j}
            + 5.41 \hat{z}
            ,
\end{align*}

which agrees with the computed value.

# Solving systems of equations

## LU decomposition

First let's create a random 5 x 5 matrix $A$ and length 5 array $b$. We'll explore how we can solve $A x = b$.

In [ ]:
a = np.random.rand(5, 5)
b = np.random.rand(5)

print("A =\n%s\n\nb = %s" % (a, b))

Let's try using SciPy's `solve` function to solve for $x$.

In [ ]:
x = linalg.solve(a, b)

print("x = %s" % x)

Let's verify the solution. Note that we don't want to use `==` here to test for equality since generally there will be some negligible error.

In [ ]:
print(np.allclose(a @ x, b))

Now we'll explicitly carry out an LU decomposition.

In [ ]:
p, l, u = linalg.lu(a)

print("P =\n%s\n\nL =\n%s\n\nU =\n%s" % (p, l, u))

Let's verify that $P L U = A$.

In [ ]:
print(np.allclose(p @ l @ u, a))

Now we'll solve two equations. The first is $L y = P^{-1} b$ for $y$, then $U x = y$ for $x$. Note that $P^{-1} = P^T$ since $P$ is a permutation matrix; this is a straightforward result from linear algebra.

In [ ]:
y_lu = linalg.solve(l, p.T @ b)
x_lu = linalg.solve(u, y_lu)

print("x = %s" % x_lu)

Note that this agrees with our earlier calculation.

## System of linear equations problems

### Resistor chain circuit

Here we need to consider the resistor chain circuit shown in the lab script.

Here we have that $V_0$ is given. For voltages $V_i$ with $i = 2, \ldots, N - 1$, Kirchoff's current law gives the result

\begin{equation}
    I_{i - 2, i} + I_{i - 1, i} + I_{i + 1, i} + I_{i + 2, i} = 0
    ,
\end{equation}

where $I_{j, i}$ is the current flowing from node $j$ to node $i$. Note that we have taken the $N + 1$th node to be the ground.

Applying Ohm's law and multiplying through by the resistance $R$ (which is the same for all resistors) we have

\begin{align}
    &\frac{1}{R} \left(\Delta V_{i - 2, i} + \Delta V_{i - 1, i} + \Delta V_{i + 1, i} + \Delta V_{i + 2, i} \right) = 0 \\
    &\Rightarrow V_{i - 2} + V_{i - 1} + V_{i + 1} + V_{i + 2} - 4 V_i = 0
    .
\end{align}

For the case of $i = 1$ we have

\begin{align}
    &I_{0, 1} + I_{2, 1} + I_{3, 1} = 0 \\
    &\Rightarrow V_0 + V_2 + V_3 - 3 V_1 = 0
    ;
\end{align}

and for $i = N$,

\begin{align}
    &I_{N - 2, N} + I_{N - 1, N} + I_{*, N} = 0 \\
    &\Rightarrow V_{N - 2} + V_{N - 1} - 3 V_N = 0
    .
\end{align}

Here, $*$ denotes the ground node.

Now let's set up a function that gives us $A$ and $b$ so that we can solve these equations.

In [ ]:
def resistor_chain_matrix(n: int, v0: float) -> Tuple[np.ndarray, np.ndarray]:
    """Returns the resistor chain matrices A and b"""
    # Construct A first
    A = 4 * np.eye(n)

    # Add off diagonals
    A = functools.reduce(lambda mat, k: mat - np.eye(n, k=k), [-2, -1, 1, 2], A)

    # Adjust the non-symmetric entries
    A[0][0] = 3
    A[n - 1, -1] = 3

    b = np.zeros(n)
    b[0] = v0
    b[1] = v0

    return (A, b)

We will test this for the $N = 6$ case where $V_0 = 4$ volts.

In [ ]:
x = linalg.solve(*resistor_chain_matrix(6, 4))

print("x = %s" % x)

Now we'll use $N = 10000$. For efficiency reasons, we want to use a "banded representation" of our matrix $A$.

In [ ]:
n = 10 ** 4
v0 = 4

# Construct banded representation of A
a = - 1 * np.ones((5, n))
a[2] = 4 * np.ones(n)
a[2][0] = 3
a[2][-1] = 3

# Get b
b = np.zeros(n)
b[0] = v0
b[1] = v0

In [ ]:
x = linalg.solve_banded((2, 2), a , b)

Now let's plot this solution.

In [ ]:
# Set up figure
_, ax = plt.subplots()

# Plot data
plt.plot(np.arange(1, n + 1), x)

# Labels
ax.set_xlabel(r"$i$")
ax.set_ylabel(r"$V_i$");

This result isn't unreasonable, but was unexpected for me that it appears to be perfectly linear.

### Resistor capacitor circuit

Now we need to solve for the voltages in a circuit involving capacitors.

Let's derive a system of equations that by applying Kirchoff's current law to the circuit diagram in the lab script.

First let's label the node with voltage $V$ the $u$ node, and the node with $0$ voltage as the $g$ node. For node $1$, we thus have

\begin{align}
    &I_{u, 1} + I_{g, 1} + I_{2, 1} = 0 \\
    &\Rightarrow \frac{V - V_1}{R_1}
        - \frac{V_1}{R_4}
        + C_1 \left( \frac{d V_2}{dt} - \frac{d V_1}{dt} \right)
        = 0 \\
    &\Rightarrow \frac{v_0 - v_1}{R_1}
        - \frac{v_1}{R_4}
        + i \omega C_1 \left( v_2 - v_1 \right)
        = 0 \\
    &\Rightarrow \left( \frac{1}{R_1} + \frac{1}{R_4} + i \omega C_1 \right) v_1
        - i \omega C_1 v_2
        + 0 v_3
        = \frac{v_0}{R_1}
    .
\end{align}

For node $2$ we have

\begin{align}
    &I_{u, 2} + I_{g, 2} + I_{1, 2} + I_{3, 2} = 0 \\
    &\Rightarrow \frac{V - V_2}{R_2}
        - \frac{V_2}{R_5}
        + C_2 \left( \frac{d V_3}{dt} - \frac{d V_2}{dt} \right)
        + C_1 \left( \frac{d V_1}{dt} - \frac{d V_2}{dt} \right)
        = 0 \\
    &\Rightarrow \frac{v_0 - v_2}{R_2}
        - \frac{v_2}{R_5}
        + i \omega C_2 \left( v_3 - v_2 \right)
        + i \omega C_1 \left( v_1 - v_2 \right)
        = 0 \\
    &\Rightarrow
        - i \omega C_1 v_1
        + \left( \frac{1}{R_2} + \frac{1}{R_5} + i \omega \left( C_1 + C_2 \right) \right) v_2
        - i \omega C_2 v_3
        = \frac{v_0}{R_2}
    .
\end{align}

And finally for node $3$ we have

\begin{align}
    &I_{u, 3} + I_{g, 3} + I_{2, 3} = 0 \\
    &\Rightarrow \frac{V - V_3}{R_3}
        - \frac{V_3}{R_6}
        + C_2 \left( \frac{d V_2}{dt} - \frac{d V_3}{dt} \right)
        = 0 \\
    &\Rightarrow \frac{v_0 - v_3}{R_3}
        - \frac{v_2}{R_6}
        + i \omega C_2 \left( v_2 - v_3 \right)
        = 0 \\
    &\Rightarrow
        0 v_1
        - i \omega C_2 v_2
        + \left( \frac{1}{R_3} + \frac{1}{R_6} + i \omega C_2 \right) v_3
        = \frac{v_0}{R_3}
    .
\end{align}

Putting our results together, we have the system of equations

\begin{align}
    \left( \frac{1}{R_1} + \frac{1}{R_4} + i \omega C_1 \right) v_1
        - i \omega C_1 v_2
        + 0 v_3
        &= \frac{v_0}{R_1}
    , \\
    - i \omega C_1 v_1
        + \left( \frac{1}{R_2} + \frac{1}{R_5} + i \omega \left( C_1 + C_2 \right) \right) v_2
        - i \omega C_2 v_3
        &= \frac{v_0}{R_2}
    , \\
    0 v_1
        - i \omega C_2 v_2
        + \left( \frac{1}{R_3} + \frac{1}{R_6} + i \omega C_2 \right) v_3
        &= \frac{v_0}{R_3}
    .
\end{align}

Let's solve this system using the values provided in the lab script.

In [ ]:
# Resistance values in ohms
r1 = 1e3
r2 = 2e3
r3 = 1e3
r4 = 2e3
r5 = 1e3
r6 = 2e3

# Capitance values in farads
c1 = 0.5e-6
c2 = 1e-6

# Voltage values
v0 = 3

# Frequency values (in Hz)
omega = 1e3

In [ ]:
# Set up A and b matrices
a = np.array(
    [
        [1 / r1 + 1 / r4 + 1j * omega * c1, -1j * omega * c1, 0],
        [-1j * omega * c1, 1 / r2 + 1 / r5 + 1j * omega * (c1 + c2), -1j * omega * c2],
        [0, -1j * omega * c2, 1 / r3 + 1 / r6 + 1j * omega * c2],
    ]
)

b = np.array([v0 / r1, v0 / r2, v0 / r3])

In [ ]:
# Solve the system
soln = linalg.solve(a, b)

In [ ]:
# Print out magnitude and phases of each voltage
for idx, v in enumerate(soln, 1):
    print("v%d: mag=%f\tphase=%+f" % (idx, np.abs(v), np.angle(v)))

# Eigensystems

To demonstrate eigenvalue solving, we'll demonstrate two different ways of finding the eigenvalues of a matrix.

In [ ]:
# Generate a random 8x8 matrix
a = np.random.rand(8, 8)

First we'll use the "determinant" method.

In [ ]:
char_poly = np.poly(a)
eigvals = np.roots(char_poly)

print("eigenvalues:\n")
print("\n".join([str(l) for l in sorted(eigvals)]))

Now we'll use a SciPy function to do the same thing.

In [ ]:
eigvals = linalg.eigvals(a)

print("eigenvalues:\n")
print("\n".join([str(l) for l in sorted(eigvals)]))

The results of both methods agree to fairly high precision.

## Power method/iteration

Now we'll demonstrate using the power method for symmetric matrices.

In [ ]:
a = np.random.rand(8, 8)
a_sym = (a + a.T) / 2

On Wikipedia there's already code for the power method, which we'll use here.

Source/credit: https://en.wikipedia.org/wiki/Power_iteration.

In [ ]:
def power_iteration(A, num_simulations: int):
    # Ideally choose a random vector
    # To decrease the chance that our vector
    # Is orthogonal to the eigenvector
    b_k = np.random.rand(A.shape[1])

    for _ in range(num_simulations):
        # calculate the matrix-by-vector product Ab
        b_k1 = np.dot(A, b_k)

        # calculate the norm
        b_k1_norm = np.linalg.norm(b_k1)

        # re normalize the vector
        b_k = b_k1 / b_k1_norm

    return b_k

Let's use this method with 5 iterations and then compare with eigenvalues found using a SciPy method.

In [ ]:
# Use power method
eigvec = power_iteration(a_sym, 5)
dom_eigval = eigvec.T @ a_sym @ eigvec

print("Largest eigenvalue (power method): %s" % dom_eigval)

In [ ]:
# Find all eigenvalues
eigvals = linalg.eigvalsh(a_sym)

print("eigenvalues:\n")
print("\n".join([str(l) for l in sorted(eigvals)]))

We see excellent agreement here (with only 5 iterations!).

## Eigenvalue/vector problems

### Community structure

Here we'll use a network analysis method to analyze the interactions between pairs of dolphins.

In [ ]:
# Read in data
data = nx.read_gml("dolphins.gml")

In [ ]:
# Get adjacency matrix
a = nx.adjacency_matrix(data)

To determine the number of nodes in the dolphin network, we just need to find the number of rows (or columns) of the adjacency matrix.

In [ ]:
num_nodes = a.shape[0]

print("num nodes: %s" % num_nodes)

Now let's construct the matrix B.

In [ ]:
# Get m and k_is
m = np.sum(a) // 2
kis = np.sum(a, axis=1)

b = a - kis @ kis.T / (2 * m)

Note that by construction, B is a symmetric matrix. Let's find its eigenvalues and eigenvectors.

In [ ]:
eigvals, eigvecs = linalg.eigh(b)

Let's plot the eigenvector corresponding to the largest eigenvalue.

In [ ]:
dom_eigvec = eigvecs[-1]

In [ ]:
# Set up figure
_, ax = plt.subplots()

# Plot data
plt.plot(range(num_nodes), dom_eigvec);

Now we'll use this eigenvector to separate our data into two groups.

In [ ]:
node_groups = np.where(dom_eigvec > 0, 1, 0)

Let's see if our groupings make sense.

In [ ]:
plt.figure()

nx.draw_networkx(data, node_color=node_groups, with_labels=False)

COMMENT HERE WHEN YOU FIGURE OUT HOW TO FIX MODEL. ABOUT "SUCCESSFULLY" FINDING TWO GROUPS.

### Normal modes of a protein: Gaussian network model

First we'll read in the data giving us the positions of the backbone carbon atoms.

In [ ]:
# Read in CSV to a dataframe
df = pd.read_csv("1rev_CAs.txt", names=["x", "y", "z"])

In [ ]:
num_points = df.shape[0]

print(num_points)

We see that there are 936 atoms in our data.

Let's plot the protein structure. In the figure below you can clearly see the binding site of the protein.

In [ ]:
# Plot
plt.figure()
ax = plt.axes(projection="3d")

ax.scatter3D(xs=df.x.values, ys=df.y.values, zs=df.z.values, s=100)

# Labels
ax.set_xlabel(r"$x$")
ax.set_ylabel(r"$y$")
ax.set_zlabel(r"$z$");

# Set the view
elev = 8.627087198515767
azim = 38.47741935483896

ax.view_init(azim=azim, elev=elev)

Now let's calculate the pairwise distances between each atom.

In [ ]:
pairwise_dists = spatial.distance.squareform(spatial.distance.pdist(df.values))

Now let's generate the Kirchoff matrix using a cutoff of $r_c = 7$ Angstrom.

In [ ]:
r_c = 7

# Calculate off diagonal elements
kirch_off_diag = np.where(pairwise_dists < r_c, -1, 0) + np.eye(num_points)

# Now do diagonal elements
kirch_diag_vec = - np.sum(kirch_off_diag, axis=1)

# Construct matrix
kirch_mat = kirch_off_diag + np.diag(kirch_diag_vec)

Now let's calculate the eigenvectors. We'll make use of the fact that the Kirchoff matrix is symmetric.

In [ ]:
eigvals, eigvecs = linalg.eigh(kirch_mat)

We'll print the first 5 eigenvalues. The first one must be approximately 0, so this is a useful check.

In [ ]:
print(eigvals[:5])

The eigenvalues seem reasonable. The first is close to zero, and the rest are positive. 

Let's redo the above scatter plot but colour according to the squared amplitude of the lowest non-zero frequency eigenvector.

In [ ]:
# Plot
plt.figure()
ax = plt.axes(projection="3d")

p = ax.scatter3D(
    xs=df.x.values, ys=df.y.values, zs=df.z.values, s=40, c=eigvecs[1] ** 2
)
plt.colorbar(p)

# Labels
ax.set_xlabel(r"$x$")
ax.set_ylabel(r"$y$")
ax.set_zlabel(r"$z$")

# Set the view
elev = 8.627087198515767
azim = 38.47741935483896

ax.view_init(azim=azim, elev=elev)

Now we'll do the same for the second lowest non-zero frequency eigenvector

In [ ]:
# Plot
plt.figure()
ax = plt.axes(projection="3d")

p = ax.scatter3D(
    xs=df.x.values, ys=df.y.values, zs=df.z.values, s=40, c=eigvecs[2] ** 2
)
plt.colorbar(p)

# Labels
ax.set_xlabel(r"$x$")
ax.set_ylabel(r"$y$")
ax.set_zlabel(r"$z$")

# Set the view
elev = 8.627087198515767
azim = 38.47741935483896

ax.view_init(azim=azim, elev=elev)

WRITE ABOUT WHY NORMAL MODE ANALYSIS WAS USEFUL